In [2]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from pandasql import sqldf

%matplotlib inline
pysql = lambda q: sqldf(q, globals())


df_box_office = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
df_box_office.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [3]:
df_box_office.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [4]:
# Connecting to database
conn = sqlite3.connect('zippedData/movies.sqlite')
# building sql cursor
cur = conn.cursor()
# Checking for existing tables
table_names = cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""").fetchall()
table_names

[('movies',), ('sqlite_sequence',), ('directors',)]

In [5]:
q = """
SELECT * 
FROM movies
"""
data = cur.execute(q).fetchall()
columns = [x[0] for x in cur.description]
movies_df = pd.DataFrame(data=data, columns=columns)
movies_df.info()

q = """
SELECT * 
FROM directors
"""
data = cur.execute(q).fetchall()
columns = [x[0] for x in cur.description]
director_df = pd.DataFrame(data=data, columns=columns)
director_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4773 entries, 0 to 4772
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              4773 non-null   int64  
 1   original_title  4773 non-null   object 
 2   budget          4773 non-null   int64  
 3   popularity      4773 non-null   int64  
 4   release_date    4773 non-null   object 
 5   revenue         4773 non-null   int64  
 6   title           4773 non-null   object 
 7   vote_average    4773 non-null   float64
 8   vote_count      4773 non-null   int64  
 9   overview        4770 non-null   object 
 10  tagline         3951 non-null   object 
 11  uid             4773 non-null   int64  
 12  director_id     4773 non-null   int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 484.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2349 entries, 0 to 2348
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ---

In [7]:
q = """
SELECT *
FROM sqlite_sequence;
"""
data = cur.execute(q).fetchall()
columns = [x[0] for x in cur.description]
seq = pd.DataFrame(data=data, columns=columns)
seq.head()

,name,seq
0,movies,48399
1,directors,7110


In [6]:
q = """
SELECT * 
FROM df_box_office
LEFT JOIN movies_df
ON df_box_office.title = movies_df.original_title;"""

test = pysql(q)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
 5   id              892 non-null    float64
 6   original_title  892 non-null    object 
 7   budget          892 non-null    float64
 8   popularity      892 non-null    float64
 9   release_date    892 non-null    object 
 10  revenue         892 non-null    float64
 11  title           892 non-null    object 
 12  vote_average    892 non-null    float64
 13  vote_count      892 non-null    float64
 14  overview        892 non-null    object 
 15  tagline         809 non-null    object 
 16  uid             892 non-null    float64
 17  director_id     892 non-null    f

In [7]:
test.sort_values('domestic_gross', ascending=False).head(20)

,title,studio,domestic_gross,foreign_gross,year,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
1872,Star Wars: The Force Awakens,BV,936700000.0,"1,131.6",2015,NaN,None,NaN,NaN,None,NaN,None,NaN,NaN,None,None,NaN,NaN
3080,Black Panther,BV,700100000.0,646900000,2018,NaN,None,NaN,NaN,None,NaN,None,NaN,NaN,None,None,NaN,NaN
3079,Avengers: Infinity War,BV,678800000.0,"1,369.5",2018,NaN,None,NaN,NaN,None,NaN,None,NaN,NaN,None,None,NaN,NaN
1873,Jurassic World,Uni.,652300000.0,"1,019.4",2015,43625.0,Jurassic World,150000000.0,418.0,2015-06-09,1.513529e+09,Jurassic World,6.5,8662.0,Twenty-two years after the events of Jurassic ...,The park is open.,135397.0,4783.0
727,Marvel's The Avengers,BV,623400000.0,895500000,2012,NaN,None,NaN,NaN,None,NaN,None,NaN,NaN,None,None,NaN,NaN
2758,Star Wars: The Last Jedi,BV,620200000.0,712400000,2017,NaN,None,NaN,NaN,None,NaN,None,NaN,NaN,None,None,NaN,NaN
3082,Incredibles 2,BV,608600000.0,634200000,2018,NaN,None,NaN,NaN,None,NaN,None,NaN,NaN,None,None,NaN,NaN
2323,Rogue One: A Star Wars Story,BV,532200000.0,523900000,2016,NaN,None,NaN,NaN,None,NaN,None,NaN,NaN,None,None,NaN,NaN
2759,Beauty and the Beast (2017),BV,504000000.0,759500000,2017,NaN,None,NaN,NaN,None,NaN,None,NaN,NaN,None,None,NaN,NaN
2324,Finding Dory,BV,486300000.0,542300000,2016,NaN,None,NaN,NaN,None,NaN,None,NaN,NaN,None,None,NaN,NaN


In [8]:
q = """
SELECT * 
FROM movies_df
WHERE original_title LIKE '%Star%'
OR original_title LIKE '%Wars%'"""

search = pysql(q)
search

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43644,Star Trek Into Darkness,190000000,78,2013-05-05,467365246,Star Trek Into Darkness,7.4,4418,When the crew of the Enterprise is called back...,Earth Will Fall,54138,4795
1,43653,Star Trek Beyond,185000000,65,2016-07-07,343471816,Star Trek Beyond,6.6,2568,The USS Enterprise crew explores the furthest ...,None,188927,4802
2,43755,Star Trek,150000000,73,2009-05-06,385680446,Star Trek,7.4,4518,The fate of the galaxy rests in the hands of b...,The future begins.,13475,4795
3,43826,Star Wars: Episode III - Revenge of the Sith,113000000,44,2005-05-17,850000000,Star Wars: Episode III - Revenge of the Sith,7.1,4116,"Years after the onset of the Clone Wars, the n...",The saga is complete.,1895,4901
4,43827,Star Wars: Episode II - Attack of the Clones,120000000,43,2002-05-15,649398328,Star Wars: Episode II - Attack of the Clones,6.4,3992,"Ten years after the invasion of Naboo, the gal...",A Jedi Shall Not Know Anger. Nor Hatred. Nor L...,1894,4901
5,43830,Star Wars: Episode I - The Phantom Menace,115000000,54,1999-05-19,924317558,Star Wars: Episode I - The Phantom Menace,6.3,4432,"Anakin Skywalker, a young slave strong with th...",Every generation has a legend. Every journey h...,1893,4901
6,43897,Starship Troopers,105000000,58,1997-11-06,121214377,Starship Troopers,6.7,1560,"Set in the future, the story follows a young s...",The only good bug is a dead bug.,563,4889
7,44178,Star Trek: Insurrection,70000000,19,1998-12-10,118000000,Star Trek: Insurrection,6.3,391,When an alien race and factions within Starfle...,The battle for paradise has begun.,200,5058
8,44188,Stardust,70000000,48,2007-08-09,135560026,Stardust,7.1,1184,In a countryside town bordering on a magical l...,This Summer A Star Falls. The Chase Begins.,2270,4830
9,44323,Starsky & Hutch,60000000,28,2004-03-05,170268750,Starsky & Hutch,5.6,644,"Join uptight David Starsky and laid-back Ken ""...",They're the man.,9384,4993


In [9]:
q = """
SELECT *, director_df.name
FROM movies_df
LEFT JOIN director_df
WHERE movies_df.director_id = director_df.id;
"""
j1 = pysql(q)
j1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4773 entries, 0 to 4772
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              4773 non-null   int64  
 1   original_title  4773 non-null   object 
 2   budget          4773 non-null   int64  
 3   popularity      4773 non-null   int64  
 4   release_date    4773 non-null   object 
 5   revenue         4773 non-null   int64  
 6   title           4773 non-null   object 
 7   vote_average    4773 non-null   float64
 8   vote_count      4773 non-null   int64  
 9   overview        4770 non-null   object 
 10  tagline         3951 non-null   object 
 11  uid             4773 non-null   int64  
 12  director_id     4773 non-null   int64  
 13  name            4773 non-null   object 
 14  id              4773 non-null   int64  
 15  gender          4773 non-null   int64  
 16  uid             4773 non-null   int64  
 17  department      4773 non-null   o

In [10]:
j1.head(10)

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id,name,id,gender,uid,department,name
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762,James Cameron,4762,2,2710,Directing,James Cameron
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763,Gore Verbinski,4763,2,1704,Directing,Gore Verbinski
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764,Sam Mendes,4764,2,39,Directing,Sam Mendes
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765,Christopher Nolan,4765,2,525,Directing,Christopher Nolan
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766,Andrew Stanton,4766,2,7,Directing,Andrew Stanton
5,43602,Spider-Man 3,258000000,115,2007-05-01,890871626,Spider-Man 3,5.9,3576,The seemingly invincible Spider-Man goes up ag...,The battle within.,559,4767,Sam Raimi,4767,2,7623,Directing,Sam Raimi
6,43603,Tangled,260000000,48,2010-11-24,591794936,Tangled,7.4,3330,When the kingdom's most wanted-and most charmi...,They're taking adventure to new lengths.,38757,4768,Byron Howard,4768,2,76595,Directing,Byron Howard
7,43604,Avengers: Age of Ultron,280000000,134,2015-04-22,1405403694,Avengers: Age of Ultron,7.3,6767,When Tony Stark tries to jumpstart a dormant p...,A New Age Has Come.,99861,4769,Joss Whedon,4769,2,12891,Directing,Joss Whedon
8,43605,Harry Potter and the Half-Blood Prince,250000000,98,2009-07-07,933959197,Harry Potter and the Half-Blood Prince,7.4,5293,"As Harry begins his sixth year at Hogwarts, he...",Dark Secrets Revealed,767,4770,David Yates,4770,2,11343,Directing,David Yates
9,43606,Batman v Superman: Dawn of Justice,250000000,155,2016-03-23,873260194,Batman v Superman: Dawn of Justice,5.7,7004,Fearing the actions of a god-like Super Hero l...,Justice or revenge,209112,4771,Zack Snyder,4771,2,15217,Directing,Zack Snyder


In [11]:
q = '''
WITH directors_names AS (
SELECT name, id
FROM director_df)

SELECT *
FROM movies_df
LEFT JOIN directors_names
ON directors_names.id = movies_df.director_id;
'''

cte = pysql(q)
cte.head()


,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id,name,id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762,James Cameron,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763,Gore Verbinski,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764,Sam Mendes,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765,Christopher Nolan,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766,Andrew Stanton,4766


In [12]:
cte.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4773 entries, 0 to 4772
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              4773 non-null   int64  
 1   original_title  4773 non-null   object 
 2   budget          4773 non-null   int64  
 3   popularity      4773 non-null   int64  
 4   release_date    4773 non-null   object 
 5   revenue         4773 non-null   int64  
 6   title           4773 non-null   object 
 7   vote_average    4773 non-null   float64
 8   vote_count      4773 non-null   int64  
 9   overview        4770 non-null   object 
 10  tagline         3951 non-null   object 
 11  uid             4773 non-null   int64  
 12  director_id     4773 non-null   int64  
 13  name            4773 non-null   object 
 14  id              4773 non-null   int64  
dtypes: float64(1), int64(8), object(6)
memory usage: 559.5+ KB
